### .env

In [1]:
%pip install dotenv

  Using cached dotenv-0.9.9-py2.py3-none-any.whl.metadata (279 bytes)
  Using cached python_dotenv-1.0.1-py3-none-any.whl.metadata (23 kB)
Using cached dotenv-0.9.9-py2.py3-none-any.whl (1.9 kB)
Using cached python_dotenv-1.0.1-py3-none-any.whl (19 kB)
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
from dotenv import load_dotenv
import os

load_dotenv()
pf_api_key = os.getenv("pf_api_key")

# Run PromptFlow

In [4]:
%pip install markdown

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import urllib.request
import json
import os
import ssl
import markdown
from IPython.display import display, HTML



def allowSelfSignedHttps(allowed):
    # bypass the server certificate verification on client side
    if allowed and not os.environ.get('PYTHONHTTPSVERIFY', '') and getattr(ssl, '_create_unverified_context', None):
        ssl._create_default_https_context = ssl._create_unverified_context

def ask_insurance_chatbot(question):
    allowSelfSignedHttps(True) # this line is needed if you use self-signed certificate in your scoring service.

    # Request data goes here
    data = {"question": question, "chat_history": []}

    body = str.encode(json.dumps(data))

    url = 'https://pf-eus-fdrydemo.eastus.inference.ml.azure.com/score'
    api_key = pf_api_key
    if not api_key:
        raise Exception("A key should be provided to invoke the endpoint")

    headers = {'Content-Type':'application/json', 'Accept': 'application/json', 'Authorization':('Bearer '+ api_key)}

    req = urllib.request.Request(url, body, headers)

    try:
        response = urllib.request.urlopen(req)
        response_text = response.read().decode("utf8", 'ignore')
        response_json = json.loads(response_text).get('answer')
        display(HTML(markdown.markdown(str(response_json))))

    except urllib.error.HTTPError as error:
        print("The request failed with status code: " + str(error.code))

        # Print the headers - they include the requert ID and the timestamp, which are useful for debugging the failure
        print(error.info())
        print(error.read().decode("utf8", 'ignore'))

In [28]:
ask_insurance_chatbot("스마트보험 가입을 위해 보험사에 알려야 하는 정보의 범위")

In [6]:
ask_insurance_chatbot("5대재해골절보험에 가입한 상태야. 한 번의 사고로 여러 개의 골절을 당했는데, 각 골절에 대해 보험금을 받을 수 있을까?")

In [57]:
ask_insurance_chatbot("재해골절보험에 가입 후 집 근처의 종합병원에 입원했어. 요양 비용을 모두 보장받을 수 있을까?")

In [58]:
ask_insurance_chatbot("스마트보험으로 코로나 진단에 대한 보장을 받을 수 있을까?")

### ⬇️ Check App Traces

[PromptFlow App Traces Workbook](https://portal.azure.com/#blade/Microsoft_Azure_WorkbooksExtension/WorkbookViewer.ReactView/ComponentId/%2Fsubscriptions%2Ff8b0b83e-4e98-41d9-90f8-7dd582a59d62%2Fresourcegroups%2Faz-fdrydemo-rg%2Fproviders%2Fmicrosoft.machinelearningservices%2Fworkspaces%2Faz-aiproject-eus-fdrydemo/ConfigurationId/%2Fsubscriptions%2Ff8b0b83e-4e98-41d9-90f8-7dd582a59d62%2FresourceGroups%2FAZ-FDRYDEMO-RG%2Fproviders%2Fmicrosoft.insights%2Fworkbooks%2F51A800C5-3BA5-45FE-AC0F-01053E44144D/WorkbookTemplateName/PromptFlow_SDK_AppTraces)

# Tracing

In [ ]:
%pip install promptflow-tracing

In [35]:
import urllib.request
import json
import os
import ssl
import markdown
from IPython.display import display, HTML

from promptflow.tracing import start_trace, trace



def allowSelfSignedHttps(allowed):
    # bypass the server certificate verification on client side
    if allowed and not os.environ.get('PYTHONHTTPSVERIFY', '') and getattr(ssl, '_create_unverified_context', None):
        ssl._create_default_https_context = ssl._create_unverified_context



@trace
def prepare_request_data(question):
    """Function to encode question into json format"""

    print("Preparing request data...")
    return str.encode(json.dumps({"question": question, "chat_history": []}))

@trace
def send_request(url, body, headers):
    """Function to send request to PromptFlow endpoint"""

    print("Sending request to PromptFlow endpoint...")
    req = urllib.request.Request(url, body, headers)
    response = urllib.request.urlopen(req)
    return response.read().decode("utf8", 'ignore')

@trace
def process_response(response_text):
    """Function to process response from PromptFlow endpoint"""

    print("Processing response...")
    response_json = json.loads(response_text).get('answer')
    display(HTML(markdown.markdown(str(response_json))))

@trace
def ask_insurance_chatbot_with_tracing(question):
    """Function to use PromptFlow chatbot service with tracing"""

    allowSelfSignedHttps(True)  # Self-signed 인증서 허용

    body = prepare_request_data(question)

    url = 'https://pf-eus-fdrydemo.eastus.inference.ml.azure.com/score'
    api_key = pf_api_key
    if not api_key:
        raise Exception("A key should be provided to invoke the endpoint")

    headers = {
        'Content-Type': 'application/json',
        'Accept': 'application/json',
        'Authorization': 'Bearer ' + api_key
    }

    try:
        response_text = send_request(url, body, headers)
        process_response(response_text)

    except urllib.error.HTTPError as error:
        print("The request failed with status code:", error.code)
        print(error.info())
        print(error.read().decode("utf8", 'ignore'))

In [39]:
start_trace()
ask_insurance_chatbot_with_tracing("손가락 골절을 당해도 보장받을 수 있을까?")

Preparing request data...
Sending request to PromptFlow endpoint...
Processing response...


## Query from Application Insights

In [ ]:
%pip install azure-monitor-query

  Using cached azure_core-1.32.0-py3-none-any.whl.metadata (39 kB)
Using cached azure_core-1.32.0-py3-none-any.whl (198 kB)
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [44]:
from dotenv import load_dotenv
import os

load_dotenv()
app_insights_app_id = os.getenv("app_insights_app_id")
app_insights_api_key = os.getenv("app_insights_api_key")

In [41]:
query = """"
AppTraces
| project AppRoleName, Message, TimeGenerated, Type
| order by TimeGenerated desc"
"""

In [45]:
import requests
import pandas as pd

url = f"https://api.applicationinsights.io/v1/apps/{app_insights_app_id}/query"

headers = {
    "x-api-key": app_insights_api_key,  # API Key authentication
    "Content-Type": "application/json"
}

params = {
    "query": query
}

try:
    response = requests.get(url, headers=headers, params=params)

    if response.status_code == 200:
        result = response.json()

        for table in result.get("tables", []):
            columns = [col["name"] for col in table["columns"]]
            rows = table["rows"]
            df = pd.DataFrame(rows, columns=columns)
            print(df)
    else:
        print(f"Error: {response.status_code}, {response.text}")

except Exception as e:
    print("Something fatal happened:", str(e))

Error: 403, {"error":{"message":"The provided credentials have insufficient access to perform the requested operation","code":"InsufficientAccessError","correlationId":"019e41ac-7ba3-4a59-80d2-f5ada3bc51dd"}}
